<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:35px;">
<b>Data Cleaning</b>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from tabulate import tabulate
from sqlalchemy import create_engine, text

In [3]:
appointments = pd.read_csv("Data/appointments.csv")
billing      = pd.read_csv("Data/billing.csv")
doctors      = pd.read_csv("Data/doctors.csv")
patients     = pd.read_csv("Data/patients.csv")
treatments   = pd.read_csv("Data/treatments.csv")

In [4]:
username = "root"
password = "password"
host     = "localhost"
port     = "3306"
database = "hospital_management"

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:25px;">
<b>1). Viewing Schema</b>

In [5]:
df = pd.read_sql("""
show tables;
"""
, con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

+---------------------------------+
| Tables_in_hospital_management   |
|---------------------------------|
| appointments                    |
| billing                         |
| doctors                         |
| patients                        |
| treatments                      |
+---------------------------------+


In [6]:
tables = ["appointments", "billing", "doctors", "patients", "treatments"]

for table in tables:
    df = pd.read_sql(f"DESCRIBE {table};", con=engine)
    print(f"\nSchema for table: {table}")
    print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))
    print("--"*40)


Schema for table: appointments
+------------------+-----------------------------------------------------+--------+-------+-----------+---------+
| Field            | Type                                                | Null   | Key   | Default   | Extra   |
|------------------+-----------------------------------------------------+--------+-------+-----------+---------|
| appointment_id   | varchar(20)                                         | NO     | PRI   |           |         |
| patient_id       | varchar(20)                                         | NO     |       |           |         |
| doctor_id        | varchar(20)                                         | NO     |       |           |         |
| appointment_date | date                                                | NO     |       |           |         |
| appointment_time | time                                                | NO     |       |           |         |
| reason_for_visit | varchar(100)                       

<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:25px;">
<b>2). Null and duplicated values in appoinments.csv</b>

In [13]:
df = pd.read_sql("""
SELECT 
    SUM(CASE WHEN appointment_id IS NULL THEN 1 ELSE 0 END) AS null_appointment_id,
    SUM(CASE WHEN patient_id IS NULL THEN 1 ELSE 0 END) AS null_patient_id,
    SUM(CASE WHEN doctor_id IS NULL THEN 1 ELSE 0 END) AS null_doctor_id,
    SUM(CASE WHEN appointment_date IS NULL THEN 1 ELSE 0 END) AS null_appointment_date,
    SUM(CASE WHEN appointment_time IS NULL THEN 1 ELSE 0 END) AS null_appointment_time,
    SUM(CASE WHEN reason_for_visit IS NULL THEN 1 ELSE 0 END) AS null_reason_for_visit,
    SUM(CASE WHEN status IS NULL THEN 1 ELSE 0 END) AS null_status
FROM appointments;
""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

+-----------------------+-------------------+------------------+-------------------------+-------------------------+-------------------------+---------------+
|   null_appointment_id |   null_patient_id |   null_doctor_id |   null_appointment_date |   null_appointment_time |   null_reason_for_visit |   null_status |
|-----------------------+-------------------+------------------+-------------------------+-------------------------+-------------------------+---------------|
|                     0 |                 0 |                0 |                       0 |                       0 |                       0 |             0 |
+-----------------------+-------------------+------------------+-------------------------+-------------------------+-------------------------+---------------+


In [12]:
df_appointments_dup = pd.read_sql("""
SELECT appointment_id, COUNT(*) AS duplicate_count
FROM appointments
GROUP BY appointment_id
HAVING duplicate_count > 1;
""", con=engine)

print("=== Duplicates: Appointments===")
print(tabulate(df_appointments_dup, headers="keys", tablefmt="psql", showindex=False), "\n")

=== Duplicates: Appointments===
+------------------+-------------------+
| appointment_id   | duplicate_count   |
|------------------+-------------------|
+------------------+-------------------+ 



<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:25px;">
<b>3). Null and duplicated values in billing.csv</b>

In [14]:
df = pd.read_sql("""
SELECT 
    SUM(CASE WHEN bill_id IS NULL THEN 1 ELSE 0 END) AS null_bill_id,
    SUM(CASE WHEN patient_id IS NULL THEN 1 ELSE 0 END) AS null_patient_id,
    SUM(CASE WHEN treatment_id IS NULL THEN 1 ELSE 0 END) AS null_treatment_id,
    SUM(CASE WHEN bill_date IS NULL THEN 1 ELSE 0 END) AS null_bill_date,
    SUM(CASE WHEN amount IS NULL THEN 1 ELSE 0 END) AS null_amount,
    SUM(CASE WHEN payment_method IS NULL THEN 1 ELSE 0 END) AS null_payment_method,
    SUM(CASE WHEN payment_status IS NULL THEN 1 ELSE 0 END) AS null_payment_status
FROM billing;
""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

+----------------+-------------------+---------------------+------------------+---------------+-----------------------+-----------------------+
|   null_bill_id |   null_patient_id |   null_treatment_id |   null_bill_date |   null_amount |   null_payment_method |   null_payment_status |
|----------------+-------------------+---------------------+------------------+---------------+-----------------------+-----------------------|
|              0 |                 0 |                   0 |                0 |             0 |                     0 |                     0 |
+----------------+-------------------+---------------------+------------------+---------------+-----------------------+-----------------------+


In [31]:
df_appointments_dup = pd.read_sql("""
SELECT bill_id, COUNT(*) AS duplicate_count
FROM billing
GROUP BY bill_id
HAVING duplicate_count > 1;
""", con=engine)

print("=== Duplicates: billing===")
print(tabulate(df_appointments_dup, headers="keys", tablefmt="psql", showindex=False), "\n")

=== Duplicates: billing===
+-----------+-------------------+
| bill_id   | duplicate_count   |
|-----------+-------------------|
+-----------+-------------------+ 



<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:25px;">
<b>4). Null and duplicated values in doctors.csv</b>

In [21]:
df = pd.read_sql("""

SELECT 
    SUM(CASE WHEN doctor_id IS NULL THEN 1 ELSE 0 END) AS null_doctor_id,
    SUM(CASE WHEN first_name IS NULL THEN 1 ELSE 0 END) AS null_first_name,
    SUM(CASE WHEN last_name IS NULL THEN 1 ELSE 0 END) AS null_last_name,
    SUM(CASE WHEN specialization IS NULL THEN 1 ELSE 0 END) AS null_specialization,
    SUM(CASE WHEN phone_number IS NULL THEN 1 ELSE 0 END) AS null_phone_number,
    SUM(CASE WHEN years_experience IS NULL THEN 1 ELSE 0 END) AS null_years_experience,
    SUM(CASE WHEN hospital_branch IS NULL THEN 1 ELSE 0 END) AS null_hospital_branch,
    SUM(CASE WHEN email IS NULL THEN 1 ELSE 0 END) AS null_email
FROM doctors;
""", con=engine)

print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

+------------------+-------------------+------------------+-----------------------+---------------------+-------------------------+------------------------+--------------+
|   null_doctor_id |   null_first_name |   null_last_name |   null_specialization |   null_phone_number |   null_years_experience |   null_hospital_branch |   null_email |
|------------------+-------------------+------------------+-----------------------+---------------------+-------------------------+------------------------+--------------|
|                0 |                 0 |                0 |                     0 |                   0 |                       0 |                      0 |            0 |
+------------------+-------------------+------------------+-----------------------+---------------------+-------------------------+------------------------+--------------+


In [30]:
df_appointments_dup = pd.read_sql("""
SELECT doctor_id, COUNT(*) AS duplicate_count
FROM doctors
GROUP BY doctor_id
HAVING duplicate_count > 1;
""", con=engine)

print("=== Duplicates: doctors===")
print(tabulate(df_appointments_dup, headers="keys", tablefmt="psql", showindex=False), "\n")

=== Duplicates: doctors===
+-------------+-------------------+
| doctor_id   | duplicate_count   |
|-------------+-------------------|
+-------------+-------------------+ 



<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:25px;">
<b>5). Null and duplicated values in patients.csv</b>

In [25]:
df = pd.read_sql("""
SELECT 
    SUM(CASE WHEN patient_id IS NULL THEN 1 ELSE 0 END) AS null_patient_id,
    SUM(CASE WHEN first_name IS NULL THEN 1 ELSE 0 END) AS null_first_name,
    SUM(CASE WHEN last_name IS NULL THEN 1 ELSE 0 END) AS null_last_name,
    SUM(CASE WHEN gender IS NULL THEN 1 ELSE 0 END) AS null_gender,
    SUM(CASE WHEN date_of_birth IS NULL THEN 1 ELSE 0 END) AS null_date_of_birth,
    SUM(CASE WHEN contact_number IS NULL THEN 1 ELSE 0 END) AS null_contact_number,
    SUM(CASE WHEN address IS NULL THEN 1 ELSE 0 END) AS null_address,
    SUM(CASE WHEN registration_date IS NULL THEN 1 ELSE 0 END) AS null_registration_date,
    SUM(CASE WHEN insurance_provider IS NULL THEN 1 ELSE 0 END) AS null_insurance_provider,
    SUM(CASE WHEN insurance_number IS NULL THEN 1 ELSE 0 END) AS null_insurance_number,
    SUM(CASE WHEN email IS NULL THEN 1 ELSE 0 END) AS null_email
    FROM patients;
""", con=engine)

print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

+-------------------+-------------------+------------------+---------------+----------------------+-----------------------+----------------+--------------------------+---------------------------+-------------------------+--------------+
|   null_patient_id |   null_first_name |   null_last_name |   null_gender |   null_date_of_birth |   null_contact_number |   null_address |   null_registration_date |   null_insurance_provider |   null_insurance_number |   null_email |
|-------------------+-------------------+------------------+---------------+----------------------+-----------------------+----------------+--------------------------+---------------------------+-------------------------+--------------|
|                 0 |                 0 |                0 |             0 |                    0 |                     0 |              0 |                        0 |                         0 |                       0 |            0 |
+-------------------+-------------------+-----------

In [29]:
df_appointments_dup = pd.read_sql("""
SELECT patient_id, COUNT(*) AS duplicate_count
FROM patients
GROUP BY patient_id
HAVING duplicate_count > 1;
""", con=engine)

print("=== Duplicates: patients===")
print(tabulate(df_appointments_dup, headers="keys", tablefmt="psql", showindex=False), "\n")

=== Duplicates: patients===
+--------------+-------------------+
| patient_id   | duplicate_count   |
|--------------+-------------------|
+--------------+-------------------+ 



<div style="background-color:#f0f8ff; padding:15px; border-radius:12px; color:#003366; font-family:Arial; font-size:25px;">
<b>6). Null and duplicated values in treatments.csv</b>

In [ ]:
# "treatments": ["treatment_id", "appointment_id", "treatment_type", "description", "cost", "treatment_date"]

In [35]:
df = pd.read_sql("""
SELECT 
    SUM(CASE WHEN treatment_id IS NULL THEN 1 ELSE 0 END) AS null_treatment_id,
    SUM(CASE WHEN appointment_id IS NULL THEN 1 ELSE 0 END) AS null_appointment_id,
    SUM(CASE WHEN treatment_type IS NULL THEN 1 ELSE 0 END) AS null_treatment_type,
    SUM(CASE WHEN description IS NULL THEN 1 ELSE 0 END) AS null_description,
    SUM(CASE WHEN treatment_date IS NULL THEN 1 ELSE 0 END) AS null_treatment_date
FROM treatments;
""", con=engine)
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))

+---------------------+-----------------------+-----------------------+--------------------+-----------------------+
|   null_treatment_id |   null_appointment_id |   null_treatment_type |   null_description |   null_treatment_date |
|---------------------+-----------------------+-----------------------+--------------------+-----------------------|
|                   0 |                     0 |                     0 |                  0 |                     0 |
+---------------------+-----------------------+-----------------------+--------------------+-----------------------+


In [36]:
df_appointments_dup = pd.read_sql("""
SELECT treatment_id, COUNT(*) AS duplicate_count
FROM treatments
GROUP BY treatment_id
HAVING duplicate_count > 1;
""", con=engine)
print("=== Duplicates: treatments===")
print(tabulate(df_appointments_dup, headers="keys", tablefmt="psql", showindex=False), "\n")

=== Duplicates: treatments===
+----------------+-------------------+
| treatment_id   | duplicate_count   |
|----------------+-------------------|
+----------------+-------------------+ 

